In [ ]:
!pip install datasets
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("scientific_papers", "pubmed")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
# Display the structure of the dataset
print(dataset)

# Show an example from the dataset
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6658
    })
})
{'article': "a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .\nin iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .\nthe prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .\nanthropometric study of elementary school students in shiraz revealed that 16% of them suff

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    cleaned_sentences = []
    for sentence in sentences:
        # Lowercase the sentence
        sentence = sentence.lower()
        # Remove digits
        sentence = re.sub(r'\d', ' ', sentence)
        # Tokenize the sentence into words
        tokens = word_tokenize(sentence)
        # Remove special characters from tokens
        tokens = [re.sub(r'\W', '', token) for token in tokens]
        # Remove stopwords
        tokens = [word for word in tokens if word and word not in stop_words]
        # Join tokens back into a single sentence
        cleaned_sentence = ' '.join(tokens)
        cleaned_sentences.append(cleaned_sentence)
    # Join cleaned sentences back into a single string
    cleaned_text = '. '.join(cleaned_sentences)
    return cleaned_text

# Apply preprocessing to the dataset
dataset = dataset.map(lambda x: {'article': preprocess_text(x['article']), 'abstract': preprocess_text(x['abstract'])})


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
# Save the preprocessed dataset to disk
dataset.save_to_disk('/mnt/data/preprocessed_pubmed_dataset')

Saving the dataset (0/4 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6633 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
from google.colab import files
import shutil

# Create a zip file of the preprocessed dataset
shutil.make_archive('/mnt/data/preprocessed_pubmed_dataset', 'zip', '/mnt/data/preprocessed_pubmed_dataset')

# Download the zip file
files.download('/mnt/data/preprocessed_pubmed_dataset.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#from datasets import load_dataset

# Load the preprocessed PubMed dataset from local drive
#dataset = load_dataset("arrow", data_files={"train": "D:\\KDD\\Ver 3\\preprocessed_pubmed_dataset\\train.arrow"})


FileNotFoundError: Unable to find 'D:\KDD\Ver 3\preprocessed_pubmed_dataset\train.arrow'